In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from sentence_transformers import SentenceTransformer
import uuid

f:\TaxMate\backend\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ============================================
# Step 1: เตรียม Qdrant และ Model สำหรับแปลงข้อความเป็น Vector
# ============================================

# เชื่อมต่อ Qdrant (ใช้ in-memory สำหรับทดสอบ)
client = QdrantClient(":memory:")

# โหลด Model สำหรับแปลงข้อความเป็น Vector
# Model นี้รองรับภาษาไทยด้วย!
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

print("เชื่อมต่อ Qdrant และโหลด Model เรียบร้อย")

เชื่อมต่อ Qdrant และโหลด Model เรียบร้อย


In [3]:
# ============================================
# Step 2: สร้าง Collection
# ============================================

collection_name = "documents"

# Model นี้สร้าง vector ขนาด 384 มิติ
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=384,  # ขนาด vector ของ model
        distance=Distance.COSINE
    )
)

print(f"สร้าง collection: {collection_name}")


สร้าง collection: documents


In [4]:
# ============================================
# Step 3: ฟังก์ชันแบ่งข้อความเป็นชิ้นเล็ก (Chunks)
# ============================================

def split_text_into_chunks(text, chunk_size=500, overlap=50):
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        
        # ถ้าไม่ใช่ chunk สุดท้าย พยายามตัดที่จุดสิ้นสุดประโยค
        if end < len(text):
            # หาจุดสิ้นสุดประโยคที่ใกล้ที่สุด
            last_period = chunk.rfind('.')
            last_newline = chunk.rfind('\n')
            cut_point = max(last_period, last_newline)
            
            if cut_point > chunk_size * 0.5:  # ถ้าจุดตัดไม่ไกลเกินไป
                chunk = chunk[:cut_point + 1]
                end = start + cut_point + 1
        
        chunks.append(chunk.strip())
        start = end - overlap  # ทับซ้อนเล็กน้อย
    
    return chunks

In [6]:
# ============================================
# Step 4: ฟังก์ชันเพิ่มไฟล์เข้า Qdrant
# ============================================

def add_text_file_to_qdrant(file_path, metadata=None):
    print(f"\nกำลังประมวลผลไฟล์: {file_path}")
    
    # อ่านไฟล์
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    print(f"   ขนาดไฟล์: {len(content)} ตัวอักษร")
    
    # แบ่งข้อความเป็นชิ้นเล็ก
    chunks = split_text_into_chunks(content, chunk_size=500, overlap=50)
    print(f"   แบ่งเป็น {len(chunks)} ชิ้น")
    
    # แปลงแต่ละชิ้นเป็น vector และเพิ่มเข้า Qdrant
    points = []
    for i, chunk in enumerate(chunks):
        # แปลงข้อความเป็น vector
        vector = model.encode(chunk).tolist()
        
        # เตรียม payload (ข้อมูลที่จะเก็บ)
        payload = {
            "text": chunk,
            "chunk_id": i,
            "file_name": file_path,
        }
        
        # เพิ่ม metadata ถ้ามี
        if metadata:
            payload.update(metadata)
        
        # สร้าง point
        point = PointStruct(
            id=str(uuid.uuid4()),  # สร้าง ID แบบ random
            vector=vector,
            payload=payload
        )
        points.append(point)
    
    # เพิ่มทั้งหมดเข้า Qdrant
    client.upsert(
        collection_name=collection_name,
        points=points
    )
    
    print(f"เพิ่ม {len(points)} chunks เข้า Qdrant เรียบร้อย\n")
    return len(points)


In [7]:
# ============================================
# Step 5: ฟังก์ชันค้นหาข้อมูล
# ============================================

def search_documents(query_text, limit=3):
    """
    ค้นหาข้อมูลที่เกี่ยวข้องกับคำถาม
    
    Args:
        query_text: คำถามหรือข้อความที่ต้องการค้นหา
        limit: จำนวนผลลัพธ์ที่ต้องการ
    """
    print(f"\nค้นหา: {query_text}")
    print("-" * 60)
    
    # แปลงคำถามเป็น vector
    query_vector = model.encode(query_text).tolist()
    
    # ค้นหาใน Qdrant
    results = client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=limit
    )
    
    # แสดงผลลัพธ์
    for i, result in enumerate(results, 1):
        print(f"\nผลลัพธ์ที่ {i}:")
        print(f"ความเกี่ยวข้อง: {result.score:.4f}")
        print(f"ไฟล์: {result.payload['file_name']}")
        print(f"เนื้อหา: {result.payload['text'][:200]}...")
        print("-" * 60)
    
    return results


In [9]:
# เพิ่มไฟล์เข้า Qdrant
add_text_file_to_qdrant("กฏหมายภาษี.txt", metadata={"category": "tax", "language": "th"})
add_text_file_to_qdrant("กฎหมายแรงงาน.txt", metadata={"category": "labor", "language": "th"})


กำลังประมวลผลไฟล์: กฏหมายภาษี.txt
   ขนาดไฟล์: 5405 ตัวอักษร
   แบ่งเป็น 13 ชิ้น
เพิ่ม 13 chunks เข้า Qdrant เรียบร้อย


กำลังประมวลผลไฟล์: กฎหมายแรงงาน.txt
   ขนาดไฟล์: 3283 ตัวอักษร
   แบ่งเป็น 8 ชิ้น
เพิ่ม 8 chunks เข้า Qdrant เรียบร้อย



8

In [ ]:
# ทดสอบค้นหา
search_documents("ค่าลดหย่อนภาษีมีอะไรบ้าง", limit=3)


ค้นหา: ค่าลดหย่อนภาษีมีอะไรบ้าง
------------------------------------------------------------

ผลลัพธ์ที่ 1:
ความเกี่ยวข้อง: 0.7782
ไฟล์: กฏหมายภาษี.txt
เนื้อหา: อัตราภาษี: 20%
คำนวณจากส่วนที่เกิน 750,000 บาท

เงินได้สุทธิ 1,000,001 - 2,000,000 บาท

อัตราภาษี: 25%
คำนวณจากส่วนที่เกิน 1,000,000 บาท

เงินได้สุทธิ 2,000,001 - 5,000,000 บาท

อัตราภาษี: 30%
คำนวณจา...
------------------------------------------------------------

ผลลัพธ์ที่ 2:
ความเกี่ยวข้อง: 0.6809
ไฟล์: กฏหมายภาษี.txt
เนื้อหา: title: กฎหมายภาษีเงินได้บุคคลธรรมดา
category: ภาษี
last_update: 2024
source: กรมสรรพากร
กฎหมายภาษีเงินได้บุคคลธรรมดา
ภาพรวม
กฎหมายภาษีเงินได้บุคคลธรรมดาเป็นกฎหมายที่กำหนดให้บุคคลธรรมดาที่มีเงินได้ต้อง...
------------------------------------------------------------


C:\Users\snail\AppData\Local\Temp\ipykernel_1440\248433527.py:20: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


[ScoredPoint(id='91bf3089-8f83-4288-a4eb-ba93d73a5686', version=0, score=0.7781538331116813, payload={'text': 'อัตราภาษี: 20%\nคำนวณจากส่วนที่เกิน 750,000 บาท\n\nเงินได้สุทธิ 1,000,001 - 2,000,000 บาท\n\nอัตราภาษี: 25%\nคำนวณจากส่วนที่เกิน 1,000,000 บาท\n\nเงินได้สุทธิ 2,000,001 - 5,000,000 บาท\n\nอัตราภาษี: 30%\nคำนวณจากส่วนที่เกิน 2,000,000 บาท\n\nเงินได้สุทธิ 5,000,001 บาทขึ้นไป\n\nอัตราภาษี: 35%\nคำนวณจากส่วนที่เกิน 5,000,000 บาท\n\nค่าลดหย่อนภาษี\nค่าลดหย่อนภาษีคือจำนวนเงินที่ผู้มีเงินได้สามารถนำมาหักออกจากเงินได้ก่อนคำนวณภาษี เพื่อลดภาระภาษีที่ต้องชำระ\nค่าลดหย่อนส่วนตัว\nค่าลดหย่อนผู้มีเงินได้', 'chunk_id': 2, 'file_name': 'กฏหมายภาษี.txt', 'category': 'tax', 'language': 'th'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='2f8b1cea-be64-4032-bf04-992852dfd193', version=0, score=0.6809276511826796, payload={'text': 'title: กฎหมายภาษีเงินได้บุคคลธรรมดา\ncategory: ภาษี\nlast_update: 2024\nsource: กรมสรรพากร\nกฎหมายภาษีเงินได้บุคคลธรรมดา\nภาพรวม\nกฎหมายภาษีเงินได้

In [17]:
search_documents("วันหยุด", limit=2)


ค้นหา: วันหยุด
------------------------------------------------------------

ผลลัพธ์ที่ 1:
ความเกี่ยวข้อง: 0.1593
ไฟล์: กฎหมายแรงงาน.txt
เนื้อหา: ทำงานครบหลายปี จะได้วันลามากขึ้น

    การเลิกจ้าง
    นายจ้างต้องบอกกล่าวล่วงหน้าหรือจ่ายค่าชดเช้อแทนการบอกกล่าว
    กรณีเลิกจ้างโดยไม่มีความผิด ต้องจ่ายค่าชดเชยตามอายุงาน...
------------------------------------------------------------

ผลลัพธ์ที่ 2:
ความเกี่ยวข้อง: 0.1348
ไฟล์: กฎหมายแรงงาน.txt
เนื้อหา: กฎหมายแรงงานไทย

    เวลาทำงาน
    ลูกจ้างต้องทำงานไม่เกิน 8 ชั่วโมงต่อวัน หรือไม่เกิน 48 ชั่วโมงต่อสัปดาห์
    กรณีทำงานล่วงเวลา ต้องจ่ายค่าล่วงเวลาอัตรา 1.5 เท่าของค่าจ้างปกติ

    วันหยุด
    ลูกจ้...
------------------------------------------------------------


C:\Users\snail\AppData\Local\Temp\ipykernel_19620\248433527.py:20: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


[ScoredPoint(id='d1c00ba8-4d27-4743-bbfc-d525ea590a0e', version=0, score=0.15929678778609435, payload={'text': 'ทำงานครบหลายปี จะได้วันลามากขึ้น\n\n    การเลิกจ้าง\n    นายจ้างต้องบอกกล่าวล่วงหน้าหรือจ่ายค่าชดเช้อแทนการบอกกล่าว\n    กรณีเลิกจ้างโดยไม่มีความผิด ต้องจ่ายค่าชดเชยตามอายุงาน', 'chunk_id': 1, 'file_name': 'กฎหมายแรงงาน.txt', 'category': 'labor', 'language': 'th'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='58b728be-74d6-48af-9445-0de26063553c', version=0, score=0.13481439480125657, payload={'text': 'กฎหมายแรงงานไทย\n\n    เวลาทำงาน\n    ลูกจ้างต้องทำงานไม่เกิน 8 ชั่วโมงต่อวัน หรือไม่เกิน 48 ชั่วโมงต่อสัปดาห์\n    กรณีทำงานล่วงเวลา ต้องจ่ายค่าล่วงเวลาอัตรา 1.5 เท่าของค่าจ้างปกติ\n\n    วันหยุด\n    ลูกจ้างมีสิทธิหยุดประจำสัปดาห์อย่างน้อยสัปดาห์ละ 1 วัน\n    มีสิทธิหยุดวันหยุดตามประเพณี 13 วัน และได้รับค่าจ้าง\n\n    วันลาพักผ่อน\n    ลูกจ้างที่ทำงานครบ 1 ปี มีสิทธิลาพักผ่อนปีละ 6 วันทำงาน\n    ลูกจ้างที่ทำงานครบหลายปี จะได้วันลามากขึ้น\n\n    การเลิกจ้าง

In [8]:
# ดูข้อมูลใน collection
print("\nข้อมูลใน Collection:")
print("-" * 60)
info = client.get_collection(collection_name)
print(f"จำนวนข้อมูลทั้งหมด: {info.points_count} chunks")
print(info)



ข้อมูลใน Collection:
------------------------------------------------------------
จำนวนข้อมูลทั้งหมด: 0 chunks
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=0 segments_count=1 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, ma